In [1]:
# dask-noise.py
print("Loading Libraries...")
import os, sys
import csv
import platform
import numpy as np
import pandas as pd
import dask.dataframe as dd
import math

Loading Libraries...


In [4]:
print("Starting code...")

print("Loading directories..")
path_data = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/pside'
path_post = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/pside-post'
path_acu = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/pside-post/acu'
path_plots = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/pside-post/plots' 
print("Loaded directories...")

Starting code...
Loading directories..
Loaded directories...


In [ ]:
print("Loading batch data...")
os.chdir(path_data)
batch_data = dd.read_csv('ss*.dat', delimiter=r"\s+", decimal='.')
print("Batch data done...")

In [ ]:
print("Calculating batch averages...")
averages = pd.DataFrame(batch_data.groupby('nodenumber').mean().compute())
print("Batch averages done...")

In [ ]:
print("Generating node coordinates...")
avg_static_p = pd.DataFrame({'pressure': averages['pressure']})
node_coords = pd.DataFrame({
    'x-coordinate': averages['x-coordinate'],
    'y-coordinate': averages['y-coordinate'],
    'z-coordinate': averages['z-coordinate']
})
print("Node coords done...")

In [ ]:
del(batch_data)
print("Batch data deleted...")

In [6]:
print("Listing files...")
filelist = sorted(os.listdir(path_data))
filelist

Listing files...


['ps-staticp-0018.dat',
 'ps-staticp-0019.dat',
 'ps-staticp-0023.dat',
 'ps-staticp-0025.dat',
 'ps-staticp-0027.dat',
 'ps-staticp-0030.dat',
 'ps-staticp-0031.dat',
 'ps-staticp-0037.dat',
 'ps-staticp-0044.dat',
 'ps-staticp-0047.dat',
 'ps-staticp-0057.dat',
 'ps-staticp-0076.dat',
 'ps-staticp-0089.dat',
 'ps-staticp-0094.dat',
 'ps-staticp-0109.dat',
 'ps-staticp-0124.dat',
 'ps-staticp-0134.dat',
 'ps-staticp-0159.dat',
 'ps-staticp-0162.dat',
 'ps-staticp-0180.dat',
 'ps-staticp-0187.dat',
 'ps-staticp-0204.dat',
 'ps-staticp-0209.dat',
 'ps-staticp-0216.dat',
 'ps-staticp-0222.dat',
 'ps-staticp-0225.dat',
 'ps-staticp-0233.dat',
 'ps-staticp-0250.dat',
 'ps-staticp-0254.dat',
 'ps-staticp-0261.dat',
 'ps-staticp-0284.dat',
 'ps-staticp-0296.dat',
 'ps-staticp-0325.dat',
 'ps-staticp-0334.dat',
 'ps-staticp-0342.dat',
 'ps-staticp-0354.dat',
 'ps-staticp-0361.dat',
 'ps-staticp-0370.dat',
 'ps-staticp-0374.dat',
 'ps-staticp-0379.dat',
 'ps-staticp-0385.dat',
 'ps-staticp-039

In [ ]:
print("Starting noise analysis loop...")
for file in filelist:
    os.chdir(path_data)
    timestep = str(os.path.basename(str(file)))[11:-4]
    time_static_p = pd.DataFrame(pd.read_csv(file, delimiter=r"\s+", header=0, usecols=["nodenumber", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
    acoustic_p = time_static_p.subtract(avg_static_p, fill_value=None)
    db = acoustic_p.apply(lambda x: 20 * np.log10(np.abs(x)/0.00002), axis=1)
    acoustic_data = pd.concat([node_coords, acoustic_p, db], axis=1)
    acoustic_data.columns = ['x-coordinate', 'y-coordinate', 'z-coordinate', 'sound-pressure', 'db-level']
    os.chdir(path_acu)
    acoustic_data.to_csv(str('sside_acu_' + str(timestep) + '.dat'), sep=',')
    print(str('sside_acu_' + str(timestep) + '.dat done...'))
print("Exiting noise analysis loop...")

In [ ]:
print("Script done, exiting.")